# Model Project: Ricardian Model of International Trade

General outline for us:

Ricardian Model
- assumptions ...
- definitions/ equations ...
- analytical solution: solve for relative prices; solving autarky, vs. trade price ratios
- numeric solution ...
- extension of the model: scale effects, transportation costs are probably easiest to implement

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

**Write out the model in equations here.** 

Make sure you explain well the purpose of the model and comment so that other students who may not have seen it before can follow.  

# Ricardian Model - General Assumptions

The Ricardian Model of International Trade is a theory that explains how gains from trade develop. The basic version assumes two countries that produce two goods. Labor is the only necessary input for production. The model describes a perfect equilibrium with perfect competition on all markets. There are no transportation costs for trade between countries. The goods that countries produce are assumed to be identical for each industry. Labor is homogeneous in a given country, but labor productivity can vary between countries. Additionally, labor is assumed to be mobile across industries in a country, but not across countries. Households (that consist of workers supplying labor to the economy) maximize utility subject to a budget constraint that depends on income. 

Model Equations - I would suggest following the International Economics Problem Set 2
visualize PPS, PPF to illustrate possible gains from trade (could also be shown at the end in further analysis)

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 